# Multilabel Prediction (Single Image)

This notebook loads the trained multilabel model (outputs/model.keras) and performs inference on one input image.

It is designed to validate the model after training and to test predictions on custom images before deploying the model (e.g., in a Flask web application).

✅ What this notebook does

After loading the model, it:

preprocesses the selected image (decode → resize → normalize)

predicts a probability score for each class

prints:

   per-class probabilities

a final decision based on configurable thresholds

✅ Target Classes

The model outputs independent probabilities for:

   - lacteos (dairy products)

   - arroz (rice)

   - frutas/verduras (fruits and vegetables)

Because this is a multilabel model, multiple classes may be detected at the same time.

✅ Decision Logic (Thresholding)

The final prediction is determined using two thresholds:

THRESH: a class is considered present if its probability ≥ THRESH

NONE_IF_MAX_BELOW: if no class passes THRESH and the maximum probability is below this value, the result is None / Not found

This logic helps reduce false positives and supports the “none” case (non-food or irrelevant images).

In [1]:
import os

def find_project_root(start="."):
    here = os.path.abspath(start)
    while True:
        if os.path.isdir(os.path.join(here, "raw")) and os.path.isdir(os.path.join(here, "dataset")):
            return here
        parent = os.path.dirname(here)
        if parent == here:
            raise RuntimeError("No encontré el root del proyecto.")
        here = parent

PROJECT_ROOT = find_project_root(".")
os.chdir(PROJECT_ROOT)

print("✅ CWD:", os.getcwd())
print("✅ Existe outputs/model.keras?:", os.path.exists("outputs/model.keras"))


✅ CWD: c:\Users\HP OMEN\Documents\GitHub\Food-Multi-Label-Classification-Pipeline-with-TensorFlow-Dataset-Builder
✅ Existe outputs/model.keras?: True



# 2) IMPORTS + CONFIG


In [2]:

import numpy as np
import tensorflow as tf

MODEL_PATH = "outputs/model.keras"
IMG_SIZE = (224, 224)

CLASSES = ["lacteos", "arroz", "frutas/verduras"]

THRESH = 0.50
NONE_IF_MAX_BELOW = 0.45


import os, glob

# tomar una imagen cualquiera del dataset
IMAGE_PATH = glob.glob("dataset/images/*.jpg")[0]
print("Usando:", IMAGE_PATH)



Usando: dataset/images\test_0000001.jpg


# 3) IMAGE LOADING + PREPROCESSING FUNCTION

In [3]:

def load_img(path):
    b = tf.io.read_file(path)
    img = tf.image.decode_image(b, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return tf.expand_dims(img, 0)


# 4) LOAD TRAINED MODEL + RUN PREDICTION

 This block performs the inference step:
   1) Load the trained multilabel model from disk
   2) Preprocess the selected image using load_img()
   3) Predict probabilities for each class
   4) Apply thresholding logic to produce a final human-readable result

 Output:
   - Prints the image path
   - Prints probability (%) per class
   - Prints the final classification result based on THRESH rules

In [4]:

model = tf.keras.models.load_model(MODEL_PATH)

x = load_img(IMAGE_PATH)
probs = model.predict(x)[0]

print("\n📌 Imagen:", IMAGE_PATH)
print("\nProbabilidades:")
for c, p in zip(CLASSES, probs):
    print(f" - {c}: {p*100:.2f}%")

labels = [c for c, p in zip(CLASSES, probs) if p >= THRESH]

if not labels and float(np.max(probs)) < NONE_IF_MAX_BELOW:
    print("\n✅ Resultado FINAL: No encontrado / No pertenece")
elif not labels:
    best = CLASSES[int(np.argmax(probs))]
    print(f"\n⚠️ Resultado FINAL: Incierto (más probable: {best})")
else:
    print("\n✅ Resultado FINAL:", ", ".join(labels))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

📌 Imagen: dataset/images\test_0000001.jpg

Probabilidades:
 - lacteos: 93.76%
 - arroz: 1.10%
 - frutas/verduras: 99.90%

✅ Resultado FINAL: lacteos, frutas/verduras


The model successfully performed inference on the input image:

Image: dataset/images/test_0000001.jpg

It produced the following class probabilities:

lacteos: 92.94%

arroz: 2.33%

frutas/verduras: 99.95%

Based on the defined threshold rules, the model detected the classes lacteos and frutas/verduras as present in the image, because both probabilities are above the classification threshold.

✅ Final Result: lacteos, frutas/verduras

This confirms that the model is capable of identifying multiple categories simultaneously, demonstrating correct behavior for real multilabel classification.